In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


NOTE: Currently, the circuit is too huge, we would do it by requesting covariance and stdev and just calculate it on verifier side instead! Other circuits in this example can run well!

In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
scales = [3]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.correlation(x, y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-64240,14942],"logrows":17,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":7872,"total_assignments":647,"total_const_size":16,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3,3],"module_sizes":{"kzg":[],"poseidon":[7872,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 11.341283798217773 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.75
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[4361588021930202802, 11452771843424695699, 17406203981540687110, 449645176536645986], [17475259421703607016, 4715985628661660007, 18118935885768207713, 749760773435915941], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14385415396251402209, 2429374486035521128, 12558163205804149944, 2583518171365219058]]], 'proof': '06ad4a01d9f6986174ae8d21b7cb17398124a680dd4f4e6d491038f50b54517a0d490c96c5cbae0175a73da188c1c3f0ebadf27495e75b0b188827e6898e9df91799fa82ce0078621cf1fd4a9f20b64bf0136bef3a7325b11eb4f50dd5ad62ff027a157c59f8395a9534d15cb067a62ca9d24b5f548cb66427f11309e03cf81f00ec900039dc99a1951cafc5bb99d76e30fd096bfc06aed61008441f1de5ce79074fc05edd6f090baa856b3733bba3d20170c36a23a8982b3bf4e079d2c8c51b0f9dbc3894394af531bb710fcd666c9385731479f30c858fac92cd9ba011bd96042e29a61d81780e0dbfdeae0b0e21b3919a1ba619a78a6fed879e5ce19dcb400510e136c28946ff006979ec4a5561b0fdeb3c1352a35e46545f3839a68221031b8039be1719bff2f5b29437d31c7ae7391db3f

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 0.75
